In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import The Necessary Packages

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import io
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn import neighbors
from sklearn.svm import SVR
from warnings import filterwarnings
filterwarnings('ignore')
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn import model_selection 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
#google colab üzerinde çalışanlar bu şekilde veri setini içeriye aktarabilir
#uploaded = files.upload()        
#df=pd.read_csv(io.BytesIO(uploaded['turkey_car_market.csv']))

In [ ]:
df=pd.read_csv("/kaggle/input/turkey-car-market-2020/turkey_car_market.csv")

In [ ]:
df.shape

In [ ]:
df.head()

Eksik veri Kontrolü yapalım

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

## Verimizi Görselleştirelim

1- Yakıt Türüne Göre Araç Fiyatlarına bakalım

In [ ]:
sns.catplot(x="Yakıt Turu", y="Fiyat", kind="bar", data=df);

2-Vites Türüne Göre Araç Fiyatları

In [ ]:
sns.catplot(x="Vites", y="Fiyat", kind="bar", data=df, palette="ch:.25");

3- Araç Durumuna Göre Fiyatlar

In [ ]:
sns.catplot(x="Durum", y="Fiyat", kind="bar", data=df);

Görüldüğü üzere 0 km araçların fiyatlarında çok büyük bir fark var. Bir de satıcıya göre fiyatları inceleyelim

In [ ]:
sns.catplot(x="Kimden", y="Fiyat", kind="bar", data=df, palette="ch:.25");

## Kolonları inceleyelim

Kolonlarlardaki benzersiz kaç kategorik veri var bunları inceleyeceğiz

In [ ]:
print(len(df['Marka'].unique()))               
markalar=df['Marka'].unique()
print(markalar)

In [ ]:
print(len(df['Arac Tip'].unique()))
car_type=df['Arac Tip'].unique()
print(car_type)

" - " ile tanımlanmış bir araç tipi var. Bu modelimiz için uygun bir veri değil. Bu veriden kaç tane olduğunu öğrenelim ve ilerleyen adımlarda üzerinde ona göre işlem yapalım

In [ ]:
len(df[df['Arac Tip']=='-'])  

In [ ]:
print(len(df['Yakıt Turu'].unique()))
yakıt_type=df['Yakıt Turu'].unique()
yakıt_type

In [ ]:
print(len(df['Vites'].unique()))
vites_type=df['Vites'].unique()
vites_type

Araç Tipi Kolonunda olduğu gibi Beygir Gücü ve CCM kolonlarında da uygun olmayan veriler mevcut. Bunlarında sayısını öğrenelim.

In [ ]:
print('Farklı CCM değer sayısı : ', len(df['CCM'].unique()),'\n')
CCM_type=df['CCM'].unique()
print(CCM_type,'\n')
print('Bilmiyorum değeri girilmiş CCM sayısı : ', len(df[df['CCM']=='Bilmiyorum']))

In [ ]:
print('Farklı Beygir gucu değer sayısı : ', len(df['Beygir Gucu'].unique()),'\n')
power=df['Beygir Gucu'].unique()
print(power,'\n')
print('Bilmiyorum değeri girilmiş Beygir Gucu sayısı : ', len(df[df['Beygir Gucu']=='Bilmiyorum']))

Araçların Model Yılı yerine yaşlarıyla işlem yapmak bize daha kolaylık sağlayacaktır. Buna Göre bir kolon oluşturalım

In [ ]:
df['car_age']=2020-df['Model Yıl']     #modelin yılı yerine yaşı ile işlem yapıcağız
df['car_age'].head()

Yaş değişkenini oluşturduğumuz için Model Yılını artık silebiliriz. Bütün araçlarında aynı yıl içinde ilan verildiğini düşünürsek işlem kolaylığı açısından İlan tarihinide silebiliriz

In [ ]:
df.drop(['Model Yıl'], axis=1,inplace=True)    #işimize yaramayan kolonları siliyoruz
df.drop(['İlan Tarihi'],axis=1,inplace=True)
df.columns

## Uygun olmayan Verileri Temizleme

Öncelikle araç tipindeki '-' ile girilmiş verileri düzenleyeceğiz. Bu şekilde az sayıda veri var. Ancak bu kolonda çeşitlilik çok fazla olduğundan dolayı bu verileri silmek yerine 'diğer' adlı bir değişkene atacağız. 

In [ ]:
df['Arac Tip']=df['Arac Tip'].str.replace('-','Diger')
len(df[df['Arac Tip']=='Diger'])

CCM kolonunda da 'Bilmiyorum' değeri girilmiş 108 tane verimiz bulunuyordu. Datasetimizin büyüklüğün düşünürsek bu değerleri silmek modelimizi çok fazla etkilemeyecektir. Çeşitlilik az olduğundan dolayı doldurma işlemide yapmamız modelimizin başka tarafa kaymasına sebep olabilir.

In [ ]:
CCM_drop=df[df['CCM']=='Bilmiyorum'].index
df.drop(CCM_drop,axis=0,inplace=True)

NOT: Beygir Gücü kolonunda 5549 adet Bilmiyorum değeri girilmiş kolon bulunuyordu. Bu kolonları En yakın komşularına göre(KNN) biraz sonra doldurma işlemi yapacağız.

In [ ]:
df.shape

Kategorik Değerleri Modelimize vermek için sayısal forma döndüreceğiz
## Label Encoder

İlk önce Araba Markalarını sayısal forma döndürüp daha sonra hangi sayının hangi modele denk geldiğini tablo şeklinde göstereceğiz

In [ ]:
l_encoder1=LabelEncoder()
df['Marka']=l_encoder1.fit_transform(df['Marka'])

In [ ]:
cars={}
car_name = list(l_encoder1.inverse_transform([i for i in range(35)]))
for i,x in enumerate(car_name):
  if i not in cars.keys():
    cars[i] =x

In [ ]:
pd.DataFrame(cars.items(), columns=['label_values', 'car_name']).head()

Kalan Diğer Değişkenlerede Label Encoder işlemini uyguluyoruz

In [ ]:
l_encoder = LabelEncoder()
columns = ['Arac Tip Grubu', 'Arac Tip','Yakıt Turu', 'Vites', 'CCM', 'Beygir Gucu', 'Renk', 'Kasa Tipi','Kimden', 'Durum']
for i in columns:
  df[i]=l_encoder.fit_transform(df[i])

In [ ]:
df.head()

In [ ]:
df.info()

Yukarıda Bahsettiğim Beygir Gücü Kolonunu doldurma zamanı geldi. 


Label Encoder işlemi uyguladığımız için şuanda ***"Bilmiyorum"*** değerine sahip verinin karşılığı 18. İlk önce Bunları NaN formuna döndürüyoruz

In [ ]:
df['Beygir Gucu']=df['Beygir Gucu'].replace(18,np.nan)  
df.isnull().sum()

Şimdi de bu Nan Değere sahip verileri KNNImputer kullanarak 5 komşu sayısına göre dolduracağız

In [ ]:
imputer = KNNImputer(n_neighbors=5)
df['Beygir Gucu']=imputer.fit_transform(df[['Beygir Gucu']])
df.isnull().sum()

In [ ]:
df['Beygir Gucu'] = round(df['Beygir Gucu'])   #doldururken float olarak bıraktığı için tam değere yuvarlıyorum

## Aykırı değerleri Silmek

In [ ]:
q1 = df["Fiyat"].quantile(0.25)
q3 = df["Fiyat"].quantile(0.75)      

IOC = q3 - q1

alt_sınır = q1 - 1.5*IOC
üst_sınır = q3 + 1.5*IOC

sınır = (df["Fiyat"] < alt_sınır) | (df["Fiyat"] > üst_sınır)
df["Aykırı_Deger"] = sınır
print('Aykırı Değer Sayısı =>\n',df["Aykırı_Deger"].value_counts())
df = df.loc[df["Aykırı_Deger"] == False]
del df["Aykırı_Deger"]

## Bağımlı ve Bağımsız Değişkenleri Ayırma

In [ ]:
y=df['Fiyat']
x=df.drop(['Fiyat'],axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)
x_train.head()

Verilerimizi sayısal forma döndürdük ancak hepsi farklı aralıklarda değer alıyor. Bunları daha düzenli şekilde modelimize vermeliyiz. Bu yüzden kolonlarımıza Standard Scaler işlemi uygulayacağız

In [ ]:
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

Veri setimizi ayırdık. şimdi modelimizi eğitme sırası geldi. ancak hangi modeli kullanarak eğiticeğimize karar vermemiz gerekiyor. bu yüzden tüm modelleri sırayla deneyen bir fonksiyon yazıyoruz. Bu fonksiyon dışardan(verdiğimiz listeden) aldığı fonksiyonları tek tek deniyor ve ekrana model score ve mean squared error değerini yazıyor. Buradan elde edeceğimiz değere göre hangi model üzerinden ilerleyeceğimizi seçeceğiz.

In [ ]:
modeller=[]
scores=[]
def compML(alg,x_train,y_train,x_test,y_test):
    model=alg().fit(x_train,y_train)
    y_pred=model.predict(x_test)
    RMSE= np.sqrt(mean_squared_error(y_test,y_pred))
    model_ismi= alg.__name__
    model_score = model.score(x_test,y_test)
    scores.append(model_score*100 if model_score > 0 else 0)
    modeller.append(model_ismi)
    print(model_ismi ," Modeli Test Hatası => ", RMSE,' |  Model Score => ', model_score*100)

models=[LGBMRegressor, Lasso,
        XGBRegressor, LinearRegression,
        GradientBoostingRegressor,
        RandomForestRegressor, ElasticNet,
        DecisionTreeRegressor, Ridge,
        MLPRegressor,
        KNeighborsRegressor, 
        SVR]

for i in models:
    compML(i,x_train,y_train,x_test,y_test) 

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.barplot(x=scores, y=modeller, palette="ch:4.5,-.7,dark=.3")
ax.set_title("Model-Skor Tablosu")
ax.set_ylabel("Modeller")
ax.set_ylabel("Score")
plt.show()

İlkel Eğitim aşamasını tamamlamış olduk. Buna Göre En iyi sonuçları LightGBM ve XGBoost gösterdi. Bu modeller üzerinden ilerleyeceğiz ve model tuning işlemi gerçekleştireceğiz. Ondan sonra son modelimize karar vereceğiz

## 1-LightGBM

In [ ]:
lgbm=LGBMRegressor()     
lgbm.fit(x_train,y_train)

lgbm_pred = lgbm.predict(x_test)

model_score = lgbm.score(x_test,y_test)
r2_skor = r2_score(y_test, lgbm_pred)
hata_skor = np.sqrt(mean_squared_error(y_test, lgbm_pred))
ev = metrics.explained_variance_score(y_test, lgbm_pred)

print("Model Score: ", model_score*100)
print("R2_skoru: ", r2_skor)
print("Hata Kare: ", hata_skor)
print("Explained Variance : ", ev)

Modelimizi optimize etmek için bir çok hiperparametre deneyeceğiz. Burada hangi değereri denemek istediğimizi bir dict formatında GridSearchCV fonksiyonuna veriyoruz

In [ ]:
lgbm_params={'learning_rate':[0.01, 0.1, 0.5],   
            'n_estimators':[200,500,1000],
            'max_depth':[5, 7, 10],
             'colsample_bytree':[0.7, 0.9, 1.0],
             'subsample': [0.4, 0.5, 0.6, 0.7]
             }
lgbm_cv_model=GridSearchCV(lgbm,lgbm_params,cv=10,n_jobs=-1,verbose=2).fit(x_train,y_train)

In [ ]:
lgbm_cv_model.best_params_

Model Tuning işlemimiz tamamlandı. En iyi parametrelere ulaştık. şimdi bunları modelimize vererek tekrardan eğiteceğiz. Böylelikle optimize final modelimize ulaşmış olacağız

In [ ]:
lgbm_tuned=LGBMRegressor(learning_rate = lgbm_cv_model.best_params_['learning_rate'],
                        max_depth = lgbm_cv_model.best_params_['max_depth'],
                        n_estimators = lgbm_cv_model.best_params_['n_estimators'],
                        colsample_bytree = lgbm_cv_model.best_params_['colsample_bytree'],
                        subsample=lgbm_cv_model.best_params_['subsample']).fit(x_train,y_train)

y_pred=lgbm_tuned.predict(x_test)

model_score = lgbm_tuned.score(x_test,y_test)
r2_skor = r2_score(y_test, y_pred)
hata_skor = np.sqrt(mean_squared_error(y_test, y_pred))
adjusted_r2_skor = 1 - (1-r2_skor)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
ev = metrics.explained_variance_score(y_test, y_pred)

print("Model scoru : ", model_score*100)
print("R2_skoru: ", r2_skor)
print("Hata Kare: ", hata_skor)
print("Adjusted_R2_skoru : ", adjusted_r2_skor)
print("Explained Variance : ", ev)

## 2-XGBoost

In [ ]:
xgb=XGBRegressor()     
xgb.fit(x_train,y_train)

xgb_pred = xgb.predict(x_test)

model_score = xgb.score(x_test,y_test)
r2_skor = r2_score(y_test, xgb_pred)
hata_skor = np.sqrt(mean_squared_error(y_test, xgb_pred))
ev = metrics.explained_variance_score(y_test, xgb_pred)

print("Model Score: ", model_score*100)
print("R2_skoru: ", r2_skor)
print("Hata Kare: ", hata_skor)
print("Explained Variance : ", ev)

In [ ]:
xgb_params = {'learning_rate':[0.01, 0.1, 0.5],
              'max_depth':[5, 7, 10],
             'colsample_bytree':[0.7, 0.9, 1.0],
             'subsample': [0.4, 0.5, 0.6, 0.7]}
xgb_cv_model=GridSearchCV(xgb,xgb_params,cv=10,n_jobs=-1,verbose=2).fit(x_train,y_train)

In [ ]:
lgbm_cv_model.best_params_

In [ ]:
xgb_tuned=XGBRegressor(learning_rate = xgb_cv_model.best_params_['learning_rate'],
                        max_depth = xgb_cv_model.best_params_['max_depth'],
                        colsample_bytree = xgb_cv_model.best_params_['colsample_bytree'],
                        subsample=xgb_cv_model.best_params_['subsample']).fit(x_train,y_train)

y_pred=xgb_tuned.predict(x_test)

model_score = xgb_tuned.score(x_test,y_test)
r2_skor = r2_score(y_test, y_pred)
hata_skor = np.sqrt(mean_squared_error(y_test, y_pred))
adjusted_r2_skor = 1 - (1-r2_skor)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
ev = metrics.explained_variance_score(y_test, y_pred)

print("Model scoru : ", model_score*100)
print("R2_skoru: ", r2_skor)
print("Hata Kare: ", hata_skor)
print("Adjusted_R2_skoru : ", adjusted_r2_skor)
print("Explained Variance : ", ev)

#### İki farklı model içinde tuning işlemini gerçekleştirdik. En iyi değerler LightGBM modelinde oluştu. O yüzden model seçimimiz LightGBM oluyor.

## tahminler ve gerçek değerler arasındaki ilişkiyi görselleştirelim 

In [ ]:
sns.scatterplot(x=y_test,y=y_pred)

In [ ]:
sns.jointplot(x=y_test, y=y_pred,  kind='reg',
                  joint_kws={'line_kws':{'color':'cyan'}})

## Gerçek Değerler ile Tahminler Arasındaki 10 aracın tablosu

In [ ]:
real_pred = pd.DataFrame({'Gerçek Fiyat': np.array(y_test).flatten(), 'Tahmini Fiyat': y_pred.flatten(),'Fark':np.array(y_test).flatten()-y_pred.flatten()})
real_pred.Fark=round(real_pred.Fark)
real_pred.head(10)

Teşekkürler :)
Linkedin : https://www.linkedin.com/in/onur-y%C4%B1lmaz-9a58a11a2/